### __Migraciones__

***

In [938]:
import pandas as pd

Carga del dataset de Migraciones.

In [939]:
Dataset_Migraciones = 'Migraciones'
Ruta = 'C:/Users/tomas/Documents/Programación/Github/Patricionog/Laboratorio de datos/Práctico 1/'
Migraciones_DB = pd.read_csv(Ruta + Dataset_Migraciones + '.csv')

Comienzo del filtrado. Lo primero es quedarnos con las filas de "TOT" en la columna de 'Migration by Gender Code'. Las filas FEM y MAL, en conjunto y sumadas, dan TOT, por ende sería información repetida e innecesaria para los fines del objetivo del estudio.

In [940]:
Migrantes = Migraciones_DB[Migraciones_DB['Migration by Gender Code'] == 'TOT']

Filtramos las columnas a las que nos importan. Además, reseteamos el índice para no generar problemas.

In [941]:
Columnas = ['Country Origin Code', 'Country Dest Code', 
            '1960 [1960]', '1970 [1970]', '1980 [1980]', '1990 [1990]', '2000 [2000]']

Migrantes = Migrantes[Columnas]
Migrantes = Migrantes.reset_index(drop=True)

Cambiamos los nombres de los atributos a los definitivos.

In [942]:
Nuevos_Nombres = {'Country Origin Code': 'id_pais_origen',
                  'Country Dest Code': 'id_pais_destino',
                  '1960 [1960]': 'cantidad_en_1960',
                  '1970 [1970]': 'cantidad_en_1970',
                  '1980 [1980]': 'cantidad_en_1980',
                  '1990 [1990]': 'cantidad_en_1990',
                  '2000 [2000]': 'cantidad_en_2000'}

Migrantes = Migrantes.rename(columns=Nuevos_Nombres)

Ya que las columnas de "cantidad_en_x" deben tener enteros, si hay strings, los convertimos en NULLS.

In [943]:
Columnas_Numericas = ['cantidad_en_1960', 'cantidad_en_1970', 'cantidad_en_1980', 
                      'cantidad_en_1990', 'cantidad_en_2000']

for Columna in Columnas_Numericas:
    Migrantes[Columna] = pd.to_numeric(Migrantes[Columna], errors='coerce')

Verificamos si hay NULLS en las columnas.

In [944]:
Count_Nulls_By_Columns = Migrantes.isnull().sum()

Llenamos los NULLS con 0. Esto es porque en la tabla había "." para cuando no había habido migraciones en ninguna de las columnas numéricas, por lo que 0 es un relleno correcto.

In [945]:
for Columna in Columnas_Numericas:
    Migrantes[Columna] = Migrantes[Columna].fillna(0)

Por las dudas, borramos espacios entre palabras (que suponemos no los hay, pero por si los hubiere perdidos por ahí) y al principio y al final.

In [946]:
Columnas_String = ['id_pais_origen', 'id_pais_destino']

for Columna in Columnas_String:
    Migrantes[Columna] = Migrantes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Creamos cinco DataFrames, cada uno con un año específico. Le agregamos una columna con el año correspondiente. Después, los filtramos: nos quedamos solo con las filas con valores mayores a 0, es decir, con filas con migrantes. Reseteamos el índice y les cambiamos el nombre. Por último, los concatenamos en el DataFrame final.

In [947]:
DataFrames_Cantidades = []

for Columna in Columnas_Numericas:
    DataFrame = Migrantes[['id_pais_origen', 'id_pais_destino', Columna]]    # Filtrado de columnas.
    DataFrame = DataFrame[DataFrame[Columna] > 0]                            # Filtrado de filas.
    DataFrame['anio'] = Columna[-4:len(Columna)+1]                           # Crear columna con el año.    
    DataFrame = DataFrame.reset_index(drop=True)                             # Resetear índices.
    DataFrame = DataFrame.rename(columns={Columna: 'migrantes'})             # Renombrar columna.
    DataFrames_Cantidades.append(DataFrame)                                  # Agregar df a lista de dfs.

Migrantes = pd.concat(DataFrames_Cantidades)

Guardamos el archivo.

In [948]:
Migrantes.to_csv('Tablas/TABLA. Migrantes.csv', index=False)

Migrantes.head(10)

,id_pais_origen,id_pais_destino,migrantes,anio
0,AFG,DZA,41.0,1960
1,AFG,ARG,6.0,1960
2,AFG,AUS,42.0,1960
3,AFG,AUT,47.0,1960
4,AFG,BHR,62.0,1960
5,AFG,BGD,151.0,1960
6,AFG,BEL,16.0,1960
7,AFG,BRA,8.0,1960
8,AFG,BGR,168.0,1960
9,AFG,BDI,1.0,1960


### __Regiones__

***

Las regiones se extraen del dataset "Sedes".

In [949]:
Dataset_Sedes = 'Sedes'
Ruta = 'C:/Users/tomas/Documents/Programación/Github/Patricionog/Laboratorio de datos/Práctico 1/'
Sedes_DB = pd.read_csv(Ruta + Dataset_Sedes + '.csv')

Arreglamos discordancias de datos entre los códigos de países en Sedes y Migraciones.

In [950]:
Sedes_DB['pais_iso_3'] = Sedes_DB['pais_iso_3'].replace('GRB', 'GBR')

Nos quedamos con las columna que nos interesa: region_geografica.

In [951]:
Regiones = pd.DataFrame(Sedes_DB['region_geografica'])

Eliminamos espacios adicionales.

In [952]:
Regiones['region_geografica'] = Regiones['region_geografica'].str.strip().str.replace(r'\s+', ' ', regex=True)

Nos quedamos con los valores únicos.

In [953]:
Regiones = Regiones.drop_duplicates()

Las ordenamos alfabéticamente y reseteamos el índice para no generar problemas.

In [954]:
Regiones = Regiones.sort_values(by='region_geografica').reset_index(drop=True)

Generamos una nueva columna: id, con números identificatorios.

In [955]:
Regiones['id'] = [x for x in range(1,len(Regiones)+1)]

Cambiamos los nombres de los atributos y ordenamos las columnas

In [956]:
Regiones = Regiones.rename(columns = {'region_geografica': 'nombre'})
Regiones = Regiones[['id', 'nombre']]

Guardamos el documento en un .csv.

In [957]:
Regiones.to_csv('Tablas/TABLA. Regiones.csv', index=False)

Regiones.head(10)

,id,nombre
0,1,AMÉRICA CENTRAL Y CARIBE
1,2,AMÉRICA DEL NORTE
2,3,AMÉRICA DEL SUR
3,4,ASIA
4,5,EUROPA CENTRAL Y ORIENTAL
5,6,EUROPA OCCIDENTAL
6,7,OCEANÍA
7,8,ÁFRICA DEL NORTE Y CERCANO ORIENTE
8,9,ÁFRICA SUBSAHARIANA


### __Países__

***

Usamos los códigos y nombres de la base de datos "Migraciones.csv", ya que son más exhaustivos que los de las otras tablas. El procedimiento para encontrar los valores únicos es:

1. Armar tres DataFrames: uno con los países de origen -código y nombre-, otro con los países de destino -código y nombre- y otro con los países de "Sedes". De esa forma obtenemos todos los países que están en una u otra columna pero no en las otras.
2. Concatenarlos verticalmente, llamando las columnas "id" y "nombre". Este DataFrame se llama "Países".
3. Se eliminan las filas con valores duplicados en la columna "id". 

In [958]:
Paises_Origen = Migraciones_DB[['Country Origin Code', 'Country Origin Name']]
Paises_Destino = Migraciones_DB[['Country Dest Code', 'Country Dest Name']]
Paises_Sedes = Sedes_DB[['pais_iso_3', 'pais_castellano']]

Renombramos las columnas para que se llamen igual.

In [959]:
Nuevos_Nombres_Origen = {'Country Origin Code': 'id',
                         'Country Origin Name': 'nombre'}

Nuevos_Nombres_Destino = {'Country Dest Code': 'id',
                          'Country Dest Name': 'nombre'}

Nuevos_Nombres_Sedes = {'pais_iso_3': 'id',
                        'pais_castellano': 'nombre'}

Paises_Origen = Paises_Origen.rename(columns = Nuevos_Nombres_Origen)
Paises_Destino = Paises_Destino.rename(columns = Nuevos_Nombres_Destino)
Paises_Sedes = Paises_Sedes.rename(columns = Nuevos_Nombres_Sedes)

Concatenamos verticalmente los dos DataFrames y creamos "Paises".

In [960]:
Paises = pd.concat([Paises_Origen, Paises_Destino, Paises_Sedes], axis=0)

Eliminamos espacios adicionales.

In [961]:
for Columna in Paises.columns:
    Paises[Columna] = Paises[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Filtramos los valores repetidos de las columna "id". Además, reseteamos el índice para no generar problemas.

In [962]:
Paises = Paises.drop_duplicates(subset=['id']).reset_index(drop=True)

Eliminamos los duplicados en la columna "pais_iso_3" en el DataFrame "Sedes". Esto es para que no se generen filas de más en el join.

In [963]:
Paises_En_Sedes = Sedes_DB.drop_duplicates(subset='pais_iso_3')

Además, eliminamos espacios adicionales.

In [964]:
for Columna in ['region_geografica', 'pais_iso_3']:
    Paises_En_Sedes[Columna] = Paises_En_Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

C:\Users\tomas\AppData\Local\Temp\ipykernel_10568\898895942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Paises_En_Sedes[Columna] = Paises_En_Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)


Agregamos el 'id_region' correspondiente a cada país. Este se extrae del correspondiente al país en el dataset "Sedes". Si no se encuentra, queda NULL.

In [965]:
Paises = pd.merge(Paises, Paises_En_Sedes, left_on='id', right_on='pais_iso_3', how='left')[['id', 'nombre', 'region_geografica']]

Hacemos un LEFT_JOIN con el DataFrame "Regiones" para extraer los id de las regiones correspondientes.

In [966]:
Paises = pd.merge(Paises, Regiones, left_on='region_geografica', right_on='nombre', how='left')

Filtramos las columnas a las que necesitamos y cambiamos nombres.

In [967]:
Paises = Paises[['id_x', 'nombre_x', 'id_y']]

Paises = Paises.rename(columns={'id_x': 'id', 'nombre_x': 'nombre', 'id_y': 'id_region'})

Contamos los NULLS y los reemplazamos con "-" (esto es opcional).

NOTA: Este punto es problemático porque hay demasiados NULLS.

In [968]:
Paises['id_region'] = Paises['id_region'].fillna('-')

Ordenamos alfabéticamente por id.

In [969]:
Paises = Paises.sort_values(by='id').reset_index(drop=True)

Guardamos el documento.

In [970]:
Paises.to_csv('Tablas/TABLA. Paises.csv', index=False)

Paises.head(10)

,id,nombre,id_region
0,ABW,Aruba,-
1,AFG,Afghanistan,-
2,AGO,Angola,9.0
3,AIA,Anguilla,-
4,ALB,Albania,-
5,AND,Andorra,-
6,ANT,Netherlands Antilles,-
7,ARE,United Arab Emirates,4.0
8,ARG,Argentina,3.0
9,ARM,Armenia,4.0


### __Sedes__

***

Se extraen del dataset "Sedes". Nos quedamos con las columnas que nos interesan y conforman los atributos.

In [971]:
Sedes = Sedes_DB[['sede_id', 'sede_desc_castellano', 'pais_iso_3']]

Cambiamos los nombres de las columnas.

In [972]:
Sedes = Sedes.rename(columns={'sede_id': 'id', 'pais_iso_3': 'id_pais', 'sede_desc_castellano': 'nombre'})

Borramos espacios adicionales.

In [973]:
for Columna in Sedes.columns:
    Sedes[Columna] = Sedes[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Ordenamos alfabéticamente por id.

In [974]:
Sedes = Sedes.sort_values(by='id').reset_index(drop=True)

Guardamos el documento.

In [975]:
Sedes.to_csv('Tablas/TABLA. Sedes.csv', index=False)

Sedes.head(10)

,id,nombre,id_pais
0,CALEG,Consulado General en Porto Alegre,BRA
1,CANTO,Consulado en Antofagasta,CHL
2,CAREN,Consulado General en Punta Arenas,CHL
3,CASUN,Consulado General en Asunción,PRY
4,CATLA,Consulado General en Atlanta,USA
5,CBARC,Consulado General y Centro de Promoción en Bar...,ESP
6,CBEIJ,Sección Consular en Beijing,CHN
7,CBELO,Consulado en Belo Horizonte,BRA
8,CBONN,Consulado en Bonn,DEU
9,CCADI,Consulado en Cádiz,ESP


### __Secciones__

***

Para armar esta tabla utilizamos el dataset "Secciones.csv".

In [976]:
Dataset_Secciones = 'Secciones'
Ruta = 'C:/Users/tomas/Documents/Programación/Github/Patricionog/Laboratorio de datos/Práctico 1/'
Secciones_DB = pd.read_csv(Ruta + Dataset_Secciones + '.csv')

Quitamos las columnas que no sirven y cambiamos los nombres de los atributos.

In [977]:
Secciones = Secciones_DB[['sede_id', 'sede_desc_castellano']]
Secciones = Secciones.rename(columns={'sede_id': 'id_sede', 'sede_desc_castellano': 'nombre'})

Borramos espacios adicionales.

In [978]:
for Columna in Secciones.columns:
    Secciones[Columna] = Secciones[Columna].str.strip().str.replace(r'\s+', ' ', regex=True)

Ordenamos alfabéticamente por id.

In [979]:
Secciones = Secciones.sort_values(by='id_sede').reset_index(drop=True)

Guardamos el documento.

In [980]:
Secciones.to_csv('Tablas/TABLA. Secciones.csv', index=False)

Secciones.head(10)

,id,nombre
0,CBARC,"Sección Cultura, Educación y Redes Sociales"
1,CBARC,Sección Administrativa
2,CBARC,Sección Consular (Asistencia al ciudadano/Visas)
3,CBARC,Sección Consular (Documentación/Asuntos previs...
4,CBARC,"Sección Comercial, Turismo e Inversiones"
5,CBARC,Sección Consular (Opción de Nacionalidad/Notar...
6,CCHIC,Sección Económica y Comercial
7,CCHIC,Sección Consular
8,CFRAN,Sección Cultural
9,CFRAN,Sección Consular


### __Atributos foráneos__

***

Vamos a chequear que todos los valores correspondientes a las columnas que son atributos foráneos existan en la columna original correspondiente. Si no existe, se agrega ese valor en el DataFrame del que se extrae el dato.

##### Migraciones

En "Migrantes" hay dos atributos foráneos: "id_pais_origen" e "id_pais_destino", que provienen de "id" de la tabla "Paises". Al haberse armado la tabla "Paises", y, en particular su columna "id", a partir de las columnas "id_pais_destino" e "id_pais_origen", así como de "pais_iso_3" de la tabla "Sedes", no es necesario verificar, ya que todos los valores provenientes de estas columnas están en la columna "id".

##### Regiones

No tiene atributos foráneos.

##### Paises

Tiene un atributo foráneo: "id_region", proveniente de la columna "id" de la tabla Regiones. El "id" es un valor ficticio que agregamos para no escribir el nombre entero en toda la tabla repetidas veces. 